How to run: <br> Simply run all the code blocks sequentially.

In [ ]:
#linking Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install python_mnist

!pip install python_mnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#load MINST images

from mnist import MNIST
import numpy as np

mnist_loader = MNIST('/content/drive/My Drive/Colab Notebooks/datasets/MNIST')
train_data, train_label = mnist_loader.load_training()
test_data, test_label = mnist_loader.load_testing()
train_data = np.array(train_data, dtype='float')/255 # norm to [0,1]
train_label = np.array(train_label, dtype='short')
test_data = np.array(test_data, dtype='float')/255 # norm to [0,1]
test_label = np.array(test_label, dtype='short')

#add small random noise to avoid matrix singularity
train_data += np.random.normal(0,0.0001,train_data.shape) 

print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


### **Model**

In [ ]:
class mySVM():
  def __init__(self, kernel='linear', optimizer='pgd', debug=0, threshold=0.001, \
               lr=1.0, max_epochs=20, batch_size=2, C=1, order=3, gamma=1.0, I=0, J=1):
    self.kernel = kernel           # kernel type
    self.optimizer = optimizer     # which optimizer is used to solve quadratic programming
    self.lr = lr                   # max learning rate in PGD
    self.max_epochs = max_epochs   # max epochs in PGD
    self.batch_size = batch_size   # size of each subset in PGD
    self.debug = debug             # whether print debugging info
    self.threshold = threshold     # threshold to filter out support vectors 
    self.label = {-1:I, 1:J}       # holding the labels

    self.C = C                     # C for the soft-margin term
    self.order = order             # power order for polynomial kernel
    self.gamma = gamma             # gamma for Gaussian RBF kernel

  # Kernel Function
  # X[N,d]: training samples;  Y[M,d]: other training samples
  # return Q[N,N]: linear kernel matrix between X and Y
  def Kernel(self, X, Y):
    if (self.kernel == 'linear'):
      K = X @ Y.T
    elif (self.kernel == 'poly'):
      K = np.power(X @ Y.T +1, self.order)
    elif (self.kernel == 'rbf'): 
      d1 = np.sum(X*X, axis=1) 
      d2 = np.sum(Y*Y, axis=1)
      K = np.outer(d1, np.ones(Y.shape[0])) + np.outer(np.ones(X.shape[0]), d2) \
          - 2 * X @ Y.T
      K = np.exp(-self.gamma * K) 
      
    return K

  # construct matrix Q from any kernel function for dual SVM optimization 
  def QuadraticMatrix(self, X, y):
    Q = np.outer(y, y) * self.Kernel(X, X) 
    return Q

      ###############################################################################################################################
      ################################################### MODIFIED CODE BELOW #######################################################
      ###############################################################################################################################

  # clipping function
  def clip(self, a, L, H):
    a_new = max(L, a)
    a_new = min(H, a_new)
    return a_new

  # use SMO to update 2 alphas in each iteration
  def SMO(self, Q, X, y):
    N = Q.shape[0]   # num of training samples
    alpha = np.zeros(N)
    epoch = 0
    while epoch < self.max_epochs:
      i = 0
      #pick i and j = i + 1
      while i < N:
        if i == N - 1:
          j = 0
        else:
          j = i + 1

        #update alpha i and j (based on my attempt of assignment 2 question 3; the formula given in my report)
        i_new = (1 - y[i]*y[j] - 2*Q[i][j]*(y[i]*y[j]*alpha[i] + alpha[j]) + 2*Q[j][j]*(alpha[i] + y[i]*y[j]*alpha[j]))/(2*(Q[i][i] - 2*Q[i][j]*y[i]*y[j] + Q[j][j]))
        if y[i] != y[j]:
          i_new = self.clip(i_new, max(0, alpha[j] - alpha[i]), min(self.C, self.C + alpha[j] - alpha[i]))
        else:
          i_new = self.clip(i_new, max(0, alpha[j] + alpha[i] - self.C), min(self.C, alpha[j] + alpha[i]))

        j_new = y[i]*y[j]*alpha[i] + alpha[j] - y[i]*y[j]*i_new

        alpha[i] = i_new
        alpha[j] = j_new

        i += 1
      epoch += 1
    return alpha
  
  # use projected gradient descent to solve quadratic program 
  # refer to Algorithm 6.5 on page 127
  # Q[N,N]: quadratic matrix;  y[N]: training labels (+1 or -1)
  def PGD(self, Q, y):
    N = Q.shape[0]   # num of training samples
    alpha = np.zeros(N)
    prev_L = 0.0

    for epoch in range(self.max_epochs):
      indices = np.random.permutation(N)  #randomly shuffle data indices
      for batch_start in range(0, N, self.batch_size):
        idx = indices[batch_start:batch_start + self.batch_size] # indices of the current subset
        alpha_s = alpha[idx]
        y_s = y[idx]

        grad_s = Q[idx,:] @ alpha - np.ones(idx.shape[0])
        proj_grad_s = grad_s - np.dot(y_s,grad_s)/np.dot(y_s, y_s)*y_s

        bound = np.zeros(idx.shape[0])
        bound[proj_grad_s < 0] = self.C 

        eta = np.min(np.abs(alpha_s-bound)/(np.abs(proj_grad_s)+0.001))

        alpha[idx] -= min(eta, self.lr) * proj_grad_s

      L = 0.5 * alpha.T @ Q @ alpha - np.sum(alpha) # objective function 
      if (L > prev_L): 
        if (self.debug>0):
          print(f'Early stopping at epoch={epoch}! ({L})')
        break
      
      if (self.debug>1):
        print(f'[PGD optimizer] epoch = {epoch}: L = {L:.5f}  (# of support vectors = {(alpha>self.threshold).sum()})')
        print(f'                 alpha: max={np.max(alpha)} min={np.min(alpha)} orthogonal constraint={np.dot(alpha,y):.2f}')

      prev_L = L

    return alpha

  # train SVM from training samples
  # X[N,d]: input features;  y[N]: output labels (+1 or -1)
  def fit(self, X, y):
    if(self.kernel != 'linear' and self.kernel != 'poly' and self.kernel != 'rbf'):
      print("Error: only linear/poly/rbf kernel is supported!")
      return

    Q = self.QuadraticMatrix(X, y)

    if self.optimizer == "smo":
      alpha = self.SMO(Q, X, y)
    elif self.optimizer == "pgd":
      alpha = self.PGD(Q, y)

    #save support vectors (pruning all data with alpha==0)
    self.X_SVs = X[alpha>self.threshold]
    self.y_SVs = y[alpha>self.threshold]
    self.alpha_SVs = alpha[alpha>self.threshold]

    if(self.kernel == 'linear'):
      self.w = (self.y_SVs * self.alpha_SVs) @ self.X_SVs

    # estimate b
    idx = np.nonzero(np.logical_and(self.alpha_SVs>self.threshold,self.alpha_SVs<self.C-self.threshold))
    if(len(idx) == 0):
      idx = np.nonzero(self.alpha_SVs>self.threshold)
    # refer to the formula on page 125 (above Figure 6.11) 
    b = self.y_SVs[idx] - (self.y_SVs * self.alpha_SVs) @ self.Kernel(self.X_SVs, self.X_SVs[idx])
    self.b = np.median(b)
    
    return 

  # use SVM from prediction (returns a vector of +1 and -1)
  # X[N,d]: input features
  def predict(self, X):
    if(self.kernel != 'linear' and self.kernel != 'poly' and self.kernel != 'rbf'):
      print("Error: only linear/poly/rbf kernel is supported!")
      return

    if(self.kernel == 'linear'):
      y = X @ self.w + self.b 
    else:
      y = (self.y_SVs * self.alpha_SVs) @ self.Kernel(self.X_SVs, X) + self.b 

    return np.sign(y)

      ###############################################################################################################################
      ################################################### MODIFIED CODE BELOW #######################################################
      ###############################################################################################################################

  # prediction function I added (returns a vector of predicted labels)
  # used for calculating testing accuracy
  def predict_label(self, X):
    if(self.kernel != 'linear' and self.kernel != 'poly' and self.kernel != 'rbf'):
      print("Error: only linear/poly/rbf kernel is supported!")
      return

    if(self.kernel == 'linear'):
      y = X @ self.w + self.b 
    else:
      y = (self.y_SVs * self.alpha_SVs) @ self.Kernel(self.X_SVs, X) + self.b 

    return np.vectorize(self.label.get)(np.sign(y))


### **Linear SVM with SMO**

In [ ]:
#model to store all 45 binary classifiers
model_list = []

for i in range(9):
  for j in range(i+1, 10):
    # prepare data
    digit_train_index = np.logical_or(train_label == i, train_label == j)
    X_train = train_data[digit_train_index]
    y_train = train_label[digit_train_index]
    digit_test_index = np.logical_or(test_label == i, test_label == j)
    X_test = test_data[digit_test_index]
    y_test = test_label[digit_test_index]

    # normalize all feature vectors to unit-length
    X_train = np.transpose (X_train.T / np.sqrt(np.sum(X_train*X_train, axis=1)))
    X_test =  np.transpose (X_test.T  / np.sqrt(np.sum(X_test*X_test, axis=1)))

    # convert labels: i => -1, j => +1
    CUTOFF = (i+j)/2 # any number between i and j
    y_train = np.sign(y_train-CUTOFF)
    y_test = np.sign(y_test-CUTOFF)

    svm = mySVM(max_epochs=20, lr=0.1, optimizer='smo', C=2, kernel='linear', debug=2, I=i, J=j)
    svm.fit(X_train,y_train)

    model_list.append(svm)

    predict = svm.predict(X_train)
    train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
    predict = svm.predict(X_test)
    test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
    print("Digits {} and {}:".format(i, j))
    print(f'MY Linear SVM: training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

#11 minutes

Digits 0 and 1:
MY Linear SVM: training accuracy=96.79%  test accuracy=97.07%
Digits 0 and 2:
MY Linear SVM: training accuracy=96.84%  test accuracy=97.02%
Digits 0 and 3:
MY Linear SVM: training accuracy=96.70%  test accuracy=97.89%
Digits 0 and 4:
MY Linear SVM: training accuracy=98.66%  test accuracy=99.29%
Digits 0 and 5:
MY Linear SVM: training accuracy=94.33%  test accuracy=94.98%
Digits 0 and 6:
MY Linear SVM: training accuracy=85.34%  test accuracy=83.95%
Digits 0 and 7:
MY Linear SVM: training accuracy=97.62%  test accuracy=98.41%
Digits 0 and 8:
MY Linear SVM: training accuracy=96.68%  test accuracy=97.19%
Digits 0 and 9:
MY Linear SVM: training accuracy=97.19%  test accuracy=97.34%
Digits 1 and 2:
MY Linear SVM: training accuracy=95.25%  test accuracy=95.71%
Digits 1 and 3:
MY Linear SVM: training accuracy=96.78%  test accuracy=97.20%
Digits 1 and 4:
MY Linear SVM: training accuracy=96.16%  test accuracy=95.94%
Digits 1 and 5:
MY Linear SVM: training accuracy=94.06%  test ac

In [ ]:
from collections import Counter

# prepare test data
X_test =  np.transpose (test_data.T  / np.sqrt(np.sum(test_data*test_data, axis=1)))
y_test = test_label

# array to hold prediction
result_arr = np.array([np.zeros(y_test.shape)])

# compute overall accuracy
for model in model_list:
  result = model.predict_label(X_test)
  result_arr = np.append(result_arr, [np.array(result)], axis=0)

#transpose and use the most frequent label in each testing sample as the final label
result_arr = (np.delete(result_arr, 0, axis=0)).T
prediction = np.array([Counter(sorted(row, reverse=True)).most_common(1)[0][0] for row in result_arr])

test_acc = np.count_nonzero(np.equal(y_test, prediction))/y_test.size
print(test_acc)

0.8312


### **Linear SVM with PGD**

In [ ]:
model_list = []
for i in range(9):
  for j in range(i+1, 10):
    # prepare data
    digit_train_index = np.logical_or(train_label == i, train_label == j)
    X_train = train_data[digit_train_index]
    y_train = train_label[digit_train_index]
    digit_test_index = np.logical_or(test_label == i, test_label == j)
    X_test = test_data[digit_test_index]
    y_test = test_label[digit_test_index]

    # normalize all feature vectors to unit-length
    X_train = np.transpose (X_train.T / np.sqrt(np.sum(X_train*X_train, axis=1)))
    X_test =  np.transpose (X_test.T  / np.sqrt(np.sum(X_test*X_test, axis=1)))

    # convert labels: i => -1, j => +1
    CUTOFF = (i+j)/2 # any number between i and j
    y_train = np.sign(y_train-CUTOFF)
    y_test = np.sign(y_test-CUTOFF)

    svm = mySVM(max_epochs=20, lr=0.1, optimizer='pgd', C=4, kernel='linear', I=i, J=j)
    svm.fit(X_train,y_train)

    model_list.append(svm)

    predict = svm.predict(X_train)
    train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
    predict = svm.predict(X_test)
    test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
    print("Digits {} and {}:".format(i, j))
    print(f'MY Linear SVM: training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

# 14 minutes

Digits 0 and 1:
MY Linear SVM: training accuracy=99.90%  test accuracy=99.91%
Digits 0 and 2:
MY Linear SVM: training accuracy=99.11%  test accuracy=99.30%
Digits 0 and 3:
MY Linear SVM: training accuracy=99.44%  test accuracy=99.75%
Digits 0 and 4:
MY Linear SVM: training accuracy=99.71%  test accuracy=99.85%
Digits 0 and 5:
MY Linear SVM: training accuracy=98.98%  test accuracy=99.09%
Digits 0 and 6:
MY Linear SVM: training accuracy=99.24%  test accuracy=99.02%
Digits 0 and 7:
MY Linear SVM: training accuracy=99.74%  test accuracy=99.60%
Digits 0 and 8:
MY Linear SVM: training accuracy=99.34%  test accuracy=99.44%
Digits 0 and 9:
MY Linear SVM: training accuracy=99.52%  test accuracy=99.30%
Digits 1 and 2:
MY Linear SVM: training accuracy=99.24%  test accuracy=99.40%
Digits 1 and 3:
MY Linear SVM: training accuracy=99.22%  test accuracy=99.63%
Digits 1 and 4:
MY Linear SVM: training accuracy=99.63%  test accuracy=99.91%
Digits 1 and 5:
MY Linear SVM: training accuracy=99.63%  test ac

In [ ]:
from collections import Counter

# prepare test data
X_test =  np.transpose (test_data.T  / np.sqrt(np.sum(test_data*test_data, axis=1)))
y_test = test_label

# array to hold prediction
result_arr = np.array([np.zeros(y_test.shape)])

# compute overall accuracy
for model in model_list:
  result = model.predict_label(X_test)
  result_arr = np.append(result_arr, [np.array(result)], axis=0)

#transpose and use the most frequent label in each testing sample as the final label
result_arr = (np.delete(result_arr, 0, axis=0)).T
prediction = np.array([Counter(sorted(row, reverse=True)).most_common(1)[0][0] for row in result_arr])

test_acc = np.count_nonzero(np.equal(y_test, prediction))/y_test.size
print(test_acc)

0.9437


### **Nonlinear SVM with SMO**

In [ ]:
c = 2
g = 9

model_list = []
for i in range(9):
  for j in range(i+1, 10):
    # prepare data
    digit_train_index = np.logical_or(train_label == i, train_label == j)
    X_train = train_data[digit_train_index]
    y_train = train_label[digit_train_index]
    digit_test_index = np.logical_or(test_label == i, test_label == j)
    X_test = test_data[digit_test_index]
    y_test = test_label[digit_test_index]

    # normalize all feature vectors to unit-length
    X_train = np.transpose (X_train.T / np.sqrt(np.sum(X_train*X_train, axis=1)))
    X_test =  np.transpose (X_test.T  / np.sqrt(np.sum(X_test*X_test, axis=1)))

    # convert labels: i => -1, j => +1
    CUTOFF = (i+j)/2 # any number between i and j
    y_train = np.sign(y_train-CUTOFF)
    y_test = np.sign(y_test-CUTOFF)

    svm = mySVM(max_epochs=20, optimizer='smo', C=c, kernel='rbf', gamma=g, debug=2, I=i, J=j)
    svm.fit(X_train,y_train)

    model_list.append(svm)

    predict = svm.predict(X_train)
    train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
    predict = svm.predict(X_test)
    test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
    print("Digits {} and {}:".format(i, j))
    print(f'MY RBF SVM (C={c}, gamma={g}): training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

# 22 minutes

Digits 0 and 1:
MY RBF SVM (C=2, gamma=9): training accuracy=97.09%  test accuracy=97.83%
Digits 0 and 2:
MY RBF SVM (C=2, gamma=9): training accuracy=99.31%  test accuracy=98.21%
Digits 0 and 3:
MY RBF SVM (C=2, gamma=9): training accuracy=99.72%  test accuracy=99.30%
Digits 0 and 4:
MY RBF SVM (C=2, gamma=9): training accuracy=99.89%  test accuracy=99.49%
Digits 0 and 5:
MY RBF SVM (C=2, gamma=9): training accuracy=99.07%  test accuracy=98.34%
Digits 0 and 6:
MY RBF SVM (C=2, gamma=9): training accuracy=99.27%  test accuracy=98.61%
Digits 0 and 7:
MY RBF SVM (C=2, gamma=9): training accuracy=99.89%  test accuracy=99.65%
Digits 0 and 8:
MY RBF SVM (C=2, gamma=9): training accuracy=99.39%  test accuracy=98.21%
Digits 0 and 9:
MY RBF SVM (C=2, gamma=9): training accuracy=99.52%  test accuracy=98.69%
Digits 1 and 2:
MY RBF SVM (C=2, gamma=9): training accuracy=97.22%  test accuracy=97.88%
Digits 1 and 3:
MY RBF SVM (C=2, gamma=9): training accuracy=97.80%  test accuracy=98.23%
Digits 1 a

In [ ]:
from collections import Counter

# prepare test data
X_test =  np.transpose (test_data.T  / np.sqrt(np.sum(test_data*test_data, axis=1)))
y_test = test_label

# array to hold prediction
result_arr = np.array([np.zeros(y_test.shape)])

# compute overall accuracy
for model in model_list:
  result = model.predict_label(X_test)
  result_arr = np.append(result_arr, [np.array(result)], axis=0)

#transpose and use the most frequent label in each testing sample as the final label
result_arr = (np.delete(result_arr, 0, axis=0)).T
prediction = np.array([Counter(sorted(row, reverse=True)).most_common(1)[0][0] for row in result_arr])

test_acc = np.count_nonzero(np.equal(y_test, prediction))/y_test.size
print(test_acc)

0.9265


### **Nonlinear SVM with PGD**

In [ ]:
c = 2
g = 9

model_list = []
for i in range(9):
  for j in range(i+1, 10):
    # prepare data
    digit_train_index = np.logical_or(train_label == i, train_label == j)
    X_train = train_data[digit_train_index]
    y_train = train_label[digit_train_index]
    digit_test_index = np.logical_or(test_label == i, test_label == j)
    X_test = test_data[digit_test_index]
    y_test = test_label[digit_test_index]

    # normalize all feature vectors to unit-length
    X_train = np.transpose (X_train.T / np.sqrt(np.sum(X_train*X_train, axis=1)))
    X_test =  np.transpose (X_test.T  / np.sqrt(np.sum(X_test*X_test, axis=1)))

    # convert labels: i => -1, j => +1
    CUTOFF = (i+j)/2 # any number between i and j
    y_train = np.sign(y_train-CUTOFF)
    y_test = np.sign(y_test-CUTOFF)

    svm = mySVM(max_epochs=20, optimizer='pgd', C=c, kernel='rbf', gamma=g, debug=0, I=i, J=j)
    svm.fit(X_train,y_train)

    model_list.append(svm)

    predict = svm.predict(X_train)
    train_acc = np.count_nonzero(np.equal(predict,y_train))/y_train.size
    predict = svm.predict(X_test)
    test_acc = np.count_nonzero(np.equal(predict,y_test))/y_test.size
    print("Digits {} and {}:".format(i, j))
    print(f'MY RBF SVM (C={c}, gamma={g}): training accuracy={100*train_acc:.2f}%  test accuracy={100*test_acc:.2f}%')

#28 minutes

Digits 0 and 1:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=99.10%
Digits 0 and 2:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=97.81%
Digits 0 and 3:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=98.49%
Digits 0 and 4:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=98.73%
Digits 0 and 5:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=97.54%
Digits 0 and 6:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=99.02%
Digits 0 and 7:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=99.05%
Digits 0 and 8:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=98.52%
Digits 0 and 9:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=98.99%
Digits 1 and 2:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=98.85%
Digits 1 and 3:
MY RBF SVM (C=2, gamma=9): training accuracy=100.00%  test accuracy=98.97%

In [ ]:
from collections import Counter

# prepare test data
X_test =  np.transpose (test_data.T  / np.sqrt(np.sum(test_data*test_data, axis=1)))
y_test = test_label

# array to hold prediction
result_arr = np.array([np.zeros(y_test.shape)])

# compute overall accuracy
for model in model_list:
  result = model.predict_label(X_test)
  result_arr = np.append(result_arr, [np.array(result)], axis=0)

#transpose and use the most frequent label in each testing sample as the final label
result_arr = (np.delete(result_arr, 0, axis=0)).T
prediction = np.array([Counter(sorted(row, reverse=True)).most_common(1)[0][0] for row in result_arr])

test_acc = np.count_nonzero(np.equal(y_test, prediction))/y_test.size
print(test_acc)

0.9631
